In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax, MaxPooling2D, BatchNormalization, ELU, Dropout, GlobalAveragePooling2D,Average, Add, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNet
import tensorflow.keras.backend as K
from keras import regularizers
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
tf.__version__

'2.14.0'

# Data Pre-Processing

Open **kyu_train.csv** file and split the games into a list.
Every row of csv: `KL0000000001,B,B[pq],W[dd],B[dp],W[pd],B[jc],...`.

Columns are:

    1. KL0000000001: Game ID
    2. B: Player's color
    3-... : Moves
    
We cropped only the moves to game list as:

In [4]:
df = open('/content/drive/Shareddrives/AICUP圍棋Dataset/kyu_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]

Create a dictionary to convert the coordinates from characters to numbers

In [5]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 4 dimensional feature map to represent the data as below:
 1. Positions of black stones: mark them as 1 and the rest of the table as 0
 2. Positions of white stones: mark them as 1 and the rest of the table as 0
 3. Empty areas of the table: mark the empty areas as 1 and occupied areas as 0
 4. The last move in the table: mark the position of the last move as 1 and the rest as 0

Target value is a number between 0-361(19\*19). Later this will be one-hot encoded.

In [6]:
def prepare_input(moves):
    x = np.zeros((19, 19, 4))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [7]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 118500, Total Moves: 27135638


The code below is run for baseline model only by using only the first 500 games from the dataset. You might need to create a data generator to use complete dataset. Otherwise your RAM might not enough to store all (If you run the code on free version of Google Colab, it will crash above 500 game samples).

In [8]:
import random

games = random.sample(games,700)
x = []
y = []
for game in games:
    moves_list = game.split(',')
    for count, move in enumerate(moves_list):
        x.append(prepare_input(moves_list[:count]))
        y.append(prepare_label(moves_list[count]))
x = np.array(x)
y = np.array(y)

In [9]:
x

array([[[[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         ...,
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         ...,
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         ...,
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]],

        ...,

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         ...,
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         ...,
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         ...,
 

In [10]:
x.shape

(158054, 19, 19, 4)

In [11]:
y.shape

(158054,)

In [12]:
y_one_hot = tf.one_hot(y, depth=19*19)

Dataset splitting: 90% Training, 10% validation

In [13]:
x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.10)

# Training

### Simple DCNN Model:

In [14]:
def create_model1():
    model = Sequential()
    model.add(Conv2D(kernel_size=19, filters=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(kernel_size=19, filters=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(kernel_size=19, filters=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(kernel_size=19, filters=3, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(kernel_size=19, filters=3, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(kernel_size=19, filters=1, padding='same', activation='relu'))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Softmax())
    model.add(Dropout(0.03))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# Create an ensemble model using Average and Lambda layers
models = [create_model1() for i in range(10)]
model_input = Input(shape=(19, 19, 4))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output)
ensemble_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
ensemble_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 19, 19, 4)]          0         []                            
                                                                                                  
 sequential (Sequential)     (None, 361)                  35127     ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 361)                  35127     ['input_1[0][0]']             
                                                                                                  
 sequential_2 (Sequential)   (None, 361)                  35127     ['input_1[0][0]']             
                                                                                              

In [16]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    "/content/drive/Shareddrives/AICUP圍棋Dataset/save_Kyu_bestModel/Kyu_best_model.h5",
    monitor = "val_loss",
    verbose = 0,
    save_best_only = True,
    save_weights_only = True,
    save_freq="epoch",
    options=None,
    initial_value_threshold=None,
)

In [ ]:
history = ensemble_model.fit(
    x = x_train,
    y = y_train,
    batch_size = 256,
    epochs = 100,
    callbacks=[model_checkpoint_callback],
    validation_data=(x_val, y_val),
)

Epoch 1/100
556/556 [==============================] - 160s 214ms/step - loss: 4.9478 - accuracy: 0.0371 - val_loss: 4.5857 - val_accuracy: 0.0435
Epoch 2/100
556/556 [==============================] - 120s 216ms/step - loss: 4.3316 - accuracy: 0.0678 - val_loss: 4.3346 - val_accuracy: 0.0774
Epoch 3/100
556/556 [==============================] - 120s 217ms/step - loss: 4.2486 - accuracy: 0.0853 - val_loss: 4.2490 - val_accuracy: 0.0887
Epoch 4/100
556/556 [==============================] - 122s 220ms/step - loss: 4.1977 - accuracy: 0.0975 - val_loss: 4.1808 - val_accuracy: 0.1071
Epoch 5/100
556/556 [==============================] - 122s 219ms/step - loss: 4.1544 - accuracy: 0.1081 - val_loss: 4.1749 - val_accuracy: 0.1063
Epoch 6/100
556/556 [==============================] - 122s 220ms/step - loss: 4.1188 - accuracy: 0.1137 - val_loss: 4.1284 - val_accuracy: 0.1163
Epoch 7/100
556/556 [==============================] - 120s 216ms/step - loss: 4.0828 - accuracy: 0.1215 - val_loss: 4

In [ ]:
model.load_weights('/content/drive/Shareddrives/AICUP圍棋Dataset/save_Kyu_bestModel/Kyu_best_model.h5')

# Testing

**PublicUpload.csv** must be in the following form:
```
KTPU0000000001,id,qr,pq,pd,ab
KTPU0000000002,ao,ab,ha,ff,qd
KTPU0000000003,qd,gd,fh,ed,fa
KTPU0000000004,pr,ba,dq,hh,aj
KTPU0000000005,ph,jh,af,df,gj
```

- Column 1: Game ID
- Column 2: Predicted Moves, up to 5 predictions for each game

The code block below is to use **kyu_test_public.csv** to predict and save the results in required form. It generates the best 5 predictions for each sample and convert them to character coordinates.

In [ ]:
def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

In [ ]:
import pandas as pd

df_ktpu = pd.read_csv('/content/drive/Shareddrives/AICUP圍棋Dataset/kyu_test_public.csv',sep='\t')
df_ktpr = pd.read_csv('/content/drive/Shareddrives/AICUP圍棋Dataset/kyu_test_private.csv',sep='\t')
df = pd.concat([df_ktpu, df_ktpu],ignore_index=False)
df = df.to_string(index=False, header=False).splitlines()

games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to PublicUpload.csv
with open('/content/drive/Shareddrives/AICUP圍棋Dataset/PublicUpload.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!

In [ ]:
print(prediction_chars)